In [25]:
import pandas as pd
import numpy as np
from random import randint
import yaml
import tabula
import os
import re

In [26]:
mak23 = tabula.read_pdf("../examples/Mak2023-2024.pdf", pages="all")

D:\Novel\Data\Python\DataAnalysis\DataAnalysisVenv\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [27]:
with open('../Notebooks/preprocessingConfig.yml', 'r') as file:
    config = yaml.safe_load(file)

In [28]:
columnNames = config['general']['columnNames']
columnNames2 = config['general']['columnNames2']
categorical_columns = config['general']['categorical_columns']
text_columns = config['general']['text_columns']
continuous_columns = config['general']['continuous_columns']

In [29]:
continuous_columns

['TOTAL WEIGHT']

In [30]:
mak23[0]['MAKERERE UIVERSITY'][5]

'U0391/673 LUKABWE DOUGLAS MARKM 2022 42 MUKONO ACC BSC. Accounting (EVE)'

In [31]:
def instTest(x):
    try: 
        if isinstance(float(x), float):
            return True
    except ValueError:
        return False

In [32]:
# Function to extract data from data2
def dataProcess(data, n):
#     if n >= 1:
    studentData = data[n].split()
    # print("Student Data: ", studentData)
    index_no = studentData[0]
    name = " ".join(studentData[1:-7])[:-1]
    gender = studentData[-8][-1]
    UACE_YEAR = int(studentData[-7])
    DistrictCode = int(studentData[-6])
    District = studentData[-5]
    CourseCode = studentData[-4]
    Course = ' '.join(studentData[-3:])
    return [index_no, name, gender, UACE_YEAR,
            DistrictCode, District, CourseCode, Course]

# Get studentData
def getStudentData(data):
    studentList = []
    for x in range(len(data)):
    #     if i > 0:
        student = dataProcess(data, x)
        studentList.append(student)
    # print(studentList)
    return studentList

# final df
def getFinalDf(pdf_read):
    df = pd.DataFrame(columns=columnNames2, data=[])
    for i in range(len(pdf_read)):
        df = pd.concat([df, getDf(i)], ignore_index=True)
    return df

# Assign dtypes to columns
# This introduces errors when used in getDf(i)
# Learning more on how to eliminate this.
def assignDtype(df):
    for column in df.columns:
        if column in categorical_columns:
              df[column] = df[column].astype("category")
        if column in text_columns:
            df[column] = df[column].astype("string")
        if column in continous_columns:
            df[column] = df[column].astype("Float32")
    return df

In [33]:
def getDf(i):
    # i = randint(0, len(mak23)-1)
    # i = 0
    # print(i)

    # First column is a common to all, drop it
    df = mak23[i]
    # print(df.head(5))
    
    columns = df.columns
    # print("columns: ", columns)
    df = df.drop(columns[0], axis=1)
    # print(df.head(10))

    # Drop columns with nan values
    # print("columns before dropping nan-columns: ", df.shape[1])
    df = df.dropna(axis="columns",
                   # thresh=3,
                   how="all",
                  )
    # print("columns after dropping: ", df.shape[1])

    # drop rows with nan if they exist
    # print("Rows before dropping: ", df.shape[0])
    df = df.dropna(axis="index",
                   # how="any",
                   thresh=3,
                  )

    # Drop rows for which Tot_wt is not float
    df = df[df[columns[-1]] != "Tot_wt"]
    # print("Weights: ", df[columns[-1]])
    # print(df.shape)
    columns = df.columns
    # print(columns)
    if len(columns) == 3:        
        
        # print(df.head(2))
        data = df[columns[1]].tolist()
        # if i == 0:
        #     data = data[1:]

        # print(data[:4])
        # data = df[columns.tolist()[1]].split()
        # print(df[df.columns[-1]])

        appData = df[df.columns[0]].tolist()
        # print(appData)
        Tot_weights = df[df.columns[-1]].tolist()
        # print("Length of Tot_weights: ", len(Tot_weights))
        # print("Samples of Tot_weights")
        # print(Tot_weights)
        temp = getStudentData(data)
        # print(temp)
        df = pd.DataFrame(columns=columnNames, data=temp)
        # print("Number of columns: ", df.shape)
        # print("Shape of temp: ", len(temp))
        # insert columns at last position
        df.insert(loc=len(df.columns),
                  column="TOTAL WEIGHT",
                  value=Tot_weights,
                 )
        df.insert(loc=0,
                  column="APPLICATION NO",
                  value=appData,
                 )
        # insert Appplication number column
        appNoData = mak23[i].loc[0]
        # df = assignDtype(df)
        return df
    if len(df.columns) == 10:
        df = df[df[columns[-1]] != "Tot_wt"]
        # Rename columns
        columns = df.columns
        df = df.rename(columns={columns[x]:columnNames2[x] for x in range(len(columns))})
        # df = assignDtype(df)
        return df
        
# Save dataframe
def save_finalDf(df):
    # current working directory
    current_path = os.getcwd()
    print(current_path)
    filepath = os.path.abspath(os.path.join(current_path, '..', 'Data'))
    filename = "Private.csv"

    print(filepath)
    saved_file = df.to_csv(os.path.join(filepath, filename), index=False)
    return saved_file

In [34]:
save_finalDf(getFinalDf(mak23))

TypeError: expected string or bytes-like object, got 'list'

In [ ]:
getFinalDf(mak23)